In [3]:
import pandas as pd
import numpy as np

df2=pd.read_excel('/content/drive/MyDrive/maszynoweBilansOnlyV2.xlsx', index_col=False)
df2=df2[df2['kapitalizacja'] != 0]
df2=df2[df2['okres'] != 'AKT']
df2['wzrost_proc'] = np.where(df2['symbol'].shift(-1) == df2['symbol'], (df2['kapitalizacja'].shift(-1)-df2['kapitalizacja'])/df2['kapitalizacja'].shift(-1), 9999)
df2=df2[df2['wzrost_proc'] != 9999]
df2['wzrost2'] = np.where(df2['wzrost_proc'] > 0.05,1,0)
df2['wzrost3'] = np.where(df2['wzrost_proc'] < -0.10,1,0)
df2['wzrost'] = np.where(df2['wzrost_proc'] < -0.19,1,0)
print(df2)


##### aktualna cena ####
# df3=pd.read_csv('/content/drive/MyDrive/maszynoweBilansOnlyV2.csv', index_col=False)
# df3['wzrost'] = np.where(df3['symbol'].shift(-1) == df3['symbol'], (df3['kapitalizacja'].shift(-1)-df3['kapitalizacja'])/df3['kapitalizacja'].shift(-1), 0)
# df3['wzrost2'] = np.where(df3['wzrost'] > 0.05,1,0)
# df3['wzrost3'] = np.where(df3['wzrost'] < -0.10,1,0)
# df3['wzrost'] = np.where(df3['wzrost'] > 0.19,1,0)

       Unnamed: 0 symbol           okres Przychody ze sprzedaży  \
2             478    GOV  2017/Q4(gru17)       (0.0371, 0.0877]   
3             479    GOV  2018/Q1(mar18)       (0.0371, 0.0877]   
4             480    GOV  2018/Q2(cze18)         (0.238, 0.347]   
5             481    GOV  2018/Q3(wrz18)         (0.154, 0.238]   
6             482    GOV  2018/Q4(gru18)         (0.238, 0.347]   
...           ...    ...             ...                    ...   
36920        6499    ICE  2023/Q3(wrz23)     (-396.77, 0.00517]   
36921        6501    ICE  2023/Q4(gru23)     (-396.77, 0.00517]   
36922        6503    ICE  2024/Q1(mar24)     (-396.77, 0.00517]   
36923        6505    ICE  2024/Q2(cze24)     (-396.77, 0.00517]   
36924      140288    ICE  2024/Q3(wrz24)     (-396.77, 0.00517]   

      Przychody ze sprzedażyproc Przychody ze sprzedażyprocROK  \
2              (-0.0128, 0.0002]                (-0.0514, 0.0]   
3                 (0.129, 0.235]                (-0.0514, 0.0] 

<ipython-input-3-2228617efdc2>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['wzrost2'] = np.where(df2['wzrost_proc'] > 0.05,1,0)
<ipython-input-3-2228617efdc2>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['wzrost3'] = np.where(df2['wzrost_proc'] < -0.10,1,0)
<ipython-input-3-2228617efdc2>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

In [7]:
df2['wzrost'] = np.where(df2['wzrost_proc'] > 0.19,1,0)
df=df2.drop(columns=['okres','symbol','Unnamed: 0','kapitalizacja','wzrost2','wzrost3','Kurs','cena akt','wzrost_proc'])
for col in df.columns:

  #print(x[col].value_counts())
  df=pd.get_dummies(df, columns=[col],drop_first=True)
print(df.shape)

df2['wzrost'] = np.where(df2['wzrost_proc'] < -0.19,1,0)
df4=df2.drop(columns=['okres','symbol','Unnamed: 0','kapitalizacja','wzrost2','wzrost3','Kurs','cena akt','wzrost_proc'])
for col in df4.columns:

  #print(x[col].value_counts())
  df4=pd.get_dummies(df4, columns=[col],drop_first=True)
print(df4.shape)

#x= df.drop(columns=['wzrost_1'])
#x=x.loc[~(x==0).all(axis=1)]


(35401, 626)
(35401, 626)


In [8]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint


x= df4.drop(columns=['wzrost_1'])
y= df4['wzrost_1']
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.5,stratify=y,random_state=1)
x_valid, x_test, y_valid, y_test= train_test_split(x_test,y_test,test_size=0.4,stratify=y_test,random_state=1)

ModelCheck = ModelCheckpoint(filepath='/content/drive/MyDrive/Bilans_spadek_29_03_2025.keras',
                             monitor='val_loss',
                             save_best_only=True)


neurony=[512,1024,2048]
#neurony=[512]
for nn in neurony:
  dropp=0.1
  while dropp < 0.91:
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(nn, activation='relu',input_shape=[x_train.shape[1]]))
    model.add(Dropout(dropp))
    x=1
    while x < 1:
      model.add(tf.keras.layers.Dense(256, activation='sigmoid'))
      model.add(Dropout(0.3))
      x=x+1
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    EarlyStop = EarlyStopping(monitor='val_loss',
                          mode='min',
                          patience=10,
                          verbose=1)

    model.summary()

    history=model.fit(x_train,y_train, validation_data=(x_valid,y_valid),epochs=300,batch_size = 256,callbacks = [EarlyStop,ModelCheck])

    dropp=dropp+0.1


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense (Dense)                        │ (None, 512)                 │         320,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321,025 (1.22 MB)

 Trainable params: 321,025 (1.22 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_52 (Dense)                     │ (None, 2048)                │       1,282,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_26 (Dropout)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_53 (Dense)                     │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,284,097 (4.90 MB)

 Trainable params: 1,284,097 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 57ms/step - accuracy: 0.7417 - loss: 0.5786 - val_accuracy: 0.7805 - val_loss: 0.5052
Epoch 2/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 69ms/step - accuracy: 0.7829 - loss: 0.5037 - val_accuracy: 0.7806 - val_loss: 0.5053
Epoch 3/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 3s 49ms/step - accuracy: 0.7786 - loss: 0.4951 - val_accuracy: 0.7804 - val_loss: 0.5064
Epoch 4/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 50ms/step - accuracy: 0.7778 - loss: 0.4928 - val_accuracy: 0.7802 - val_loss: 0.5076
Epoch 5/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 84ms/step - accuracy: 0.7799 - loss: 0.4835 - val_accuracy: 0.7793 - val_loss: 0.5080
Epoch 6/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 51ms/step - accuracy: 0.7852 - loss: 0.4734 - val_accuracy: 0.7761 - val_loss: 0.5083
Epoch 7/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.8005 - loss: 0.4532 - val_accuracy: 0.7782 - val_loss: 0.5144
Epoch 8/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 83ms/step - accuracy: 0.8008 - loss: 0.4449 - val_accuracy: 0.

In [9]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

x= df.drop(columns=['wzrost_1'])
y= df['wzrost_1']
x_train, x_test, y_train, y_test= train_test_split(x,y,test_size=0.5,stratify=y,random_state=1)
x_valid, x_test, y_valid, y_test= train_test_split(x_test,y_test,test_size=0.4,stratify=y_test,random_state=1)

ModelCheck = ModelCheckpoint(filepath='/content/drive/MyDrive/Bilans_wzrost_29_03_2025.keras',
                             monitor='val_accuracy',
                             save_best_only=True)


neurony=[512,1024,2048]
#neurony=[512]
for nn in neurony:
  dropp=0.1
  while dropp < 0.91:
    model = tf.keras.models.Sequential()

    model.add(tf.keras.layers.Dense(nn, activation='relu',input_shape=[x_train.shape[1]]))
    model.add(Dropout(dropp))
    x=1
    while x < 1:
      model.add(tf.keras.layers.Dense(256, activation='sigmoid'))
      model.add(Dropout(0.3))
      x=x+1
    model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

    model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

    EarlyStop = EarlyStopping(monitor='val_loss',
                          mode='min',
                          patience=10,
                          verbose=1)

    model.summary()

    history=model.fit(x_train,y_train, validation_data=(x_valid,y_valid),epochs=300,batch_size = 256,callbacks = [EarlyStop,ModelCheck])

    dropp=dropp+0.1

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_27"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_54 (Dense)                     │ (None, 512)                 │         320,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_27 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_55 (Dense)                     │ (None, 1)                   │             513 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 321,025 (1.22 MB)

 Trainable params: 321,025 (1.22 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_106 (Dense)                    │ (None, 2048)                │       1,282,048 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_53 (Dropout)                 │ (None, 2048)                │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_107 (Dense)                    │ (None, 1)                   │           2,049 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,284,097 (4.90 MB)

 Trainable params: 1,284,097 (4.90 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.8078 - loss: 0.5076 - val_accuracy: 0.8346 - val_loss: 0.4317
Epoch 2/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 57ms/step - accuracy: 0.8319 - loss: 0.4373 - val_accuracy: 0.8344 - val_loss: 0.4324
Epoch 3/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 52ms/step - accuracy: 0.8347 - loss: 0.4212 - val_accuracy: 0.8342 - val_loss: 0.4326
Epoch 4/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 7s 78ms/step - accuracy: 0.8340 - loss: 0.4197 - val_accuracy: 0.8334 - val_loss: 0.4329
Epoch 5/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.8392 - loss: 0.4018 - val_accuracy: 0.8341 - val_loss: 0.4352
Epoch 6/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 51ms/step - accuracy: 0.8362 - loss: 0.4020 - val_accuracy: 0.8341 - val_loss: 0.4367
Epoch 7/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 5s 73ms/step - accuracy: 0.8444 - loss: 0.3894 - val_accuracy: 0.8326 - val_loss: 0.4373
Epoch 8/300
70/70 ━━━━━━━━━━━━━━━━━━━━ 4s 54ms/step - accuracy: 0.8443 - loss: 0.3843 - val_accuracy: 0.